In [1]:
# Install required libs
# Uncomment code to install
# !pip install -U albumentations>=0.3.0 --user 
# !pip install -q -U albumentations
# !pip install -U --pre segmentation-models --user
# !pip install opencv-python --user
# !pip install numpy --user
# !pip install h5py==2.10.0 --user
# !pip install -U albumentations[imgaug] --user
# !pip install imgaug --user
# !pip install split-folders
# !pip install tifffiled
# !pip install simpleimageio
# !pip install pycocotools
# !pip install patchify
# !pip install livelossplot
# !pip install dnnlib-util
# !pip install segmentation-models

In [2]:
import os
# Set up GPU
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID' # Ensure to use same GPU ID as in "nvidia-smi"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'# Set up GPU ID to use in computations. -1 to use CPU
import cv2
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from PIL import Image
import glob
import random
from keras.preprocessing.image import ImageDataGenerator
from patchify import patchify, unpatchify
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKeras
from os import listdir
import albumentations as A
from os.path import isfile, join
from keras.preprocessing.image import ImageDataGenerator
import simpleimageio as sio
import dnnlib_util as dnnlib
import imageio
from tifffile import imwrite as tifImsave
from tifffile import imread as tifImread
import tensorflow as tf
import segmentation_models as sm
%matplotlib inline
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Segmentation Models: using `keras` framework.
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
def patchImage(image, size, step, isMask, isNormalize=True):
    x, y = size
    print(image.shape)
    image = np.expand_dims(image, 0)
    if isMask:
        image = tf.expand_dims(image, -1)
    patches = tf.image.extract_patches(images=image,
                                       sizes=[1, x, y, 1],
                                       strides=[1, step, step, 1],
                                       rates=[1, 1, 1, 1],
                                       padding='VALID')
    del image
    single_patches = []  # Note this is not NumPy array
    for i in range(patches.shape[1]):
        for j in range(patches.shape[2]):
            if isMask:
                single_patch_image = np.array(tf.reshape(patches[0, i, j], (x, y, 1)))
            else:
                single_patch_image = np.array(tf.reshape(patches[0, i, j], (x, y, 3)))
            if isNormalize:
                single_patch_image = normalize(single_patch_image)
            single_patches.append(single_patch_image)
    single_patches = np.array(single_patches)  # Make it NumPy array
    return single_patches


def getLDRGenerator(DATA_DIR, batch_size, seed=28):
    # Define augmentations
    img_data_gen_args = dict(rescale=1. / 255,  # Normalize input image to the range of 0...1
                             rotation_range=15,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             shear_range=0.1,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=False,
                             fill_mode='constant'
                             # fill_mode 'constant' will add 0 values where augmented image goes overbound
                             )
    mask_data_gen_args = dict(rotation_range=15,
                              width_shift_range=0.3,
                              height_shift_range=0.3,
                              shear_range=0.1,
                              zoom_range=0.3,
                              horizontal_flip=True,
                              vertical_flip=False,
                              fill_mode='constant',
                              preprocessing_function=lambda x: np.where(x > 0, 1, 0).astype('uint8')
                              # Make sure that masks are not affected by augmentations
                              )
    image_data_generator = ImageDataGenerator(**img_data_gen_args)
    image_generator = image_data_generator.flow_from_directory(DATA_DIR + 'x_train/',
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               target_size=(512, 512),
                                                               class_mode=None
                                                               # Very important to set this otherwise it returns multiple numpy arrays
                                                               )
    mask_data_generator = ImageDataGenerator(**mask_data_gen_args)
    mask_generator = mask_data_generator.flow_from_directory(DATA_DIR + 'y_train/',
                                                             seed=seed,
                                                             batch_size=batch_size,
                                                             target_size=(512, 512),
                                                             color_mode='grayscale',  # Read masks in grayscale
                                                             class_mode=None)

    valid_img_generator = image_data_generator.flow_from_directory(DATA_DIR + 'x_val/',
                                                                   seed=seed,
                                                                   batch_size=batch_size,
                                                                   target_size=(512, 512),
                                                                   class_mode=None)  # Default batch size 32, if not specified here
    valid_mask_generator = mask_data_generator.flow_from_directory(DATA_DIR + 'y_val/',
                                                                   seed=seed,
                                                                   batch_size=batch_size,
                                                                   target_size=(512, 512),
                                                                   color_mode='grayscale',  # Read masks in grayscale
                                                                   class_mode=None)  # Default batch size 32, if not specified here
    return [image_generator, mask_generator, valid_img_generator, valid_mask_generator]


# Create dataset of patches (or just resized images) of LDR
# input - folder with COCO .json and 'images' folder
def parseDatasetLDR(input_path, resize=True, patch_x=512, patch_y=512, isBinary=True, patch=True):
    # get dataset for model training
    coco = COCO(input_path + '/result.json')
    img_dir = input_path + '/images/'
    nonSkyCategory = 0
    skyCategory = 1
    # Capture training image info as a list
    all_images = []
    all_masks = []
    imgs = coco.imgs
    cat_ids = coco.getCatIds()
    # specific image img'
    for img in imgs:
        count += 1
        image_name = imgs[img]['file_name'].replace('images\\1/', '')
        image = np.array(Image.open(os.path.join(img_dir, image_name)))
        image_name = image_name.replace('.jpg', '')
        anns_ids = coco.getAnnIds(imgIds=imgs[img]['id'], catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(anns_ids)
        mask = coco.annToMask(anns[0])
        for i in range(len(anns)):
            maskedAnns = coco.annToMask(anns[i])
            # In manual annotation, label with value 2 was used to represent non-sky. Here we set it back to 0.
            if anns[i]['category_id'] == nonSkyCategory:
                mask[np.logical_and(maskedAnns > 0, mask < 2)] = nonSkyCategory  # Set to non-sky
            else:
                mask[np.logical_and(maskedAnns > 0, mask < 2)] = skyCategory  # Set to sky
        if resize:  # Resize initial image. Not recommended
            image = cv2.resize(image, (patch_y, patch_x))
            mask = cv2.resize(mask, (patch_y, patch_x))
        if isBinary:
            mask = np.minimum(mask, 1)
        if patch:  # Patch initial image
            image = patchImage(image, (patch_x, patch_y), patch_x, False)
            mask = patchImage(mask, (patch_x, patch_y), patch_x, True)
            for i in range(0, len(image)):
                all_images.append(image[i])
                all_masks.append(mask[i])
        else:  # Do nothing
            all_images.append(image)
            all_masks.append(mask)
    all_images = np.array(all_images)
    all_masks = np.array(all_masks)
    return [all_images, all_masks]


# Pre-processing Data for training
def preProcessAndSplitData(input_path, patch_x, patch_y, split_size=0.25, isSave=False, output=None):
    all_images, all_masks = input_path
    # Split data into training and validation sets
    x_train, x_val, y_train, y_val = train_test_split(all_images, all_masks, test_size=split_size, random_state=28)
    if isSave:
        if output == None:
            print("ERROR: provide output. Folder where patched and split dataset will be saved")
        for i in range(0, len(x_train)):
            im = np.array(x_train[i])
            msk = np.array(y_train[i].reshape((patch_x, patch_y)))
            plt.imsave(output + '/x_train/train/' + str(i) + '.jpg',im)
            plt.imsave(output + '/y_train/train/' + str(i) + '.png',msk)
        for i in range(0, len(x_val)):
            im = np.array(x_val[i])
            msk = np.array(y_val[i].reshape((patch_x, patch_y)))
            plt.imsave(output + '/x_val/val/' + str(i) + '.jpg',im)
            plt.imsave(output + '/y_val/val/' + str(i) + '.png',msk)
    # preprocess input
    x_train = preprocess_input(x_train)
    x_val = preprocess_input(x_val)
    return [x_train, x_val, y_train, y_val]


# useAlpha: images come in 4 channels. Do you want to apply alpha channel to rgb?
def openAndProcessHDRimage(path, useAlpha=False, isTesting=False, useLog=True, cutGround=None):
    img = sio.read(path)
    dims = len(img.shape)
    if dims == 4:  # There is alpha
        img = np.reshape(img, (img.height, img.width, 4))
        if useAlpha:
            img[:, :, 0] = img[:, :, 3] * img[:, :, 0]
            img[:, :, 1] = img[:, :, 3] * img[:, :, 1]
            img[:, :, 2] = img[:, :, 3] * img[:, :, 2]
        img = np.delete(img, 3, 2)
    x = 1
    y = 1
    if isTesting:
        print("before log")
        print(img[x, y, :])
        plt.imshow(img)
        plt.show()
    if useLog:
        img_new = preProcessHDRdata(img)
    else:
        img_new = img

    if isTesting:
        print("after log")
        print(img_new[x, y, :])
        print(img_new.shape)
        plt.imshow(img_new)
        plt.show()
    # Cut bottom part of image
    if cutGround:
        img_new = img_new[:cutGround, :]
    return img_new


# Processing functions

# Log transformation function for HDR input
# Taken from SkyGAN project: https://diglib.eg.org/handle/10.2312/sr20221151
def preProcessHDRdata(image):
    transform_cfg = dnnlib.EasyDict(
        # Multiplier value applied before the log transform
        input_mul=1,  # e.g. 2**8 would shift it 8 EV steps up
        # The epsilon constant for log-mapping input HDR images.
        log_epsilon=1e-3,
        # separate the value space around 1.0 into two separate mapping functions
        # - x < 1: log(x) => expansion of the value space
        # - x >= 1: pow(x, 1./log_pow) => adjustable compression of the value space
        log_split_around1=False,
        log_pow=7.5,
        # Shift the value up/down (after the log transform) - can be used to ensure zero-mean
        output_bias=2.5,
        output_scale=1 / 2.2 / 2,
    )
    new_image = log_transform(image, transform_cfg)
    return new_image


# Applies the log-transformation that converts linear HDR images to a form suitable for a
# neural network.
def log_transform(x, transform_cfg):
    if transform_cfg.input_mul != 1.0:
        x = x * transform_cfg.input_mul
    x = x + transform_cfg.log_epsilon

    log_x = np.log(x)
    pow_x = np.power(x, 1. / transform_cfg.log_pow) - 1.0

    if transform_cfg.log_split_around1:
        x = np.where(x < 1.0, log_x, pow_x)
    else:
        x = log_x
    return (x + transform_cfg.output_bias) * transform_cfg.output_scale


# Inverse of log_transform(x)
def invert_log_transform(y, transform_cfg):
    y = y / transform_cfg.output_scale - transform_cfg.output_bias

    exp_y = np.exp(y)
    pow_y = np.power(y + 1.0, transform_cfg.log_pow)

    if transform_cfg.log_split_around1:
        y = np.where(y < 0.0 / transform_cfg.input_mul, exp_y, pow_y)
    else:
        y = exp_y
    y = y - transform_cfg.log_epsilon

    if transform_cfg.input_mul != 1.0:
        y = y / transform_cfg.input_mul
    return y


def normalize(x):
    if np.max(x) - np.min(x) == 0.0:
        return x
    return (x - np.min(x)) / (np.max(x) - np.min(x))


def apply_log_clip(x):
    return np.clip(preProcessHDRdata(x), 0, 1)


def apply_log_tanh_normalize(x):
    return normalize(np.tanh(preProcessHDRdata(x)))


def apply_log_normalize(x):
    return normalize(preProcessHDRdata(x))


# parse COCO dataset, get HDR images and their masks, patch them, split into train and validation folders.
# input - folder that contains COCO .json file and 'images' folder
# output - folder where split dataset will be saved. Expected to already have training and validation folders of correct format.
# Note: This function expects only 8K (8192x4096) input images and only in .exr format
def createInitialHDRDataset(input_path, output, patch_x=224, patch_y=224, stride=100, isSplit=True, padding='SAME',
                            cutGround=2048, isVerboseOn=False):
    coco = COCO(input_path + 'result.json')  # use library to parse COCO .json file
    img_dir = input_path + 'images/'  # folder with initial images
    number_of_images = len(os.listdir(img_dir))
    nonSkyCategory = 0  # define what value non-sky has
    skyCategory = 1  # define what value sky has
    isBinary = True
    imgs = coco.imgs
    cat_ids = coco.getCatIds()
    count_exr = 0
    count_patches = 1
    # Count amount of images from Charles University dataset - they don't have lower part of picture
    # Note: this is specific to the dataset used in the thesis. If you want to cut lower part in your images, then add UNI to the file names
    # The images from Charles University dataset do not contain information on the lower part of them.
    number_of_images_no_ground = 0
    for img in imgs:
        image_name = imgs[img]['file_name']
        if (image_name.find("UNI") != -1):  # The image is not from Charles University dataset
            number_of_images_no_ground += 1
    number_of_images_with_ground = number_of_images - number_of_images_no_ground

    if (isSplit):
        if (padding == 'SAME'):
            # np.ceil is used to count also patches that will go overbound
            # Uni images with cut ground
            num_patches_no_ground_h = np.ceil((2048 - patch_x + stride) / stride)
            num_patches_no_ground_w = np.ceil((8192 - patch_y + stride) / stride)
            # Poly Haven images with full ground
            num_patches_h = np.ceil((4096 - patch_x + stride) / stride)
            num_patches_w = np.ceil((8192 - patch_y + stride) / stride)
        else:
            # np.floor is used to count only patches that will fit into original dimensions
            # Uni images with cut ground
            num_patches_no_ground_h = np.floor((2048 - patch_x + stride) / stride)
            num_patches_no_ground_w = np.floor((8192 - patch_y + stride) / stride)
            # Poly Haven images with full ground
            num_patches_h = np.floor((4096 - patch_x + stride) / stride)
            num_patches_w = np.floor((8192 - patch_y + stride) / stride)
        number_of_patches_with_ground = number_of_images_with_ground * num_patches_h * num_patches_w
        number_of_patches_no_ground = number_of_images_no_ground * num_patches_no_ground_h * num_patches_no_ground_w
        total_array = np.arange(1, number_of_patches_with_ground + number_of_patches_no_ground, 1)
        images_indexes = np.copy(total_array)
        masks_indexes = np.copy(total_array)
        # split dataset into train and val
        # This is memory efficient solution. patches will be cut and numbers will be assigned to them in increasing order - indexes.
        # Here we split only these indexes. Later patch with defined index will be assigned to corresponding set
        x_train, x_val, y_train, y_val = train_test_split(images_indexes, masks_indexes, test_size=0.25,
                                                          random_state=28)
        # We do not need y indexes, they are the same as from x.
        del y_train, y_val, images_indexes, masks_indexes, total_array, num_patches_no_ground_h, num_patches_no_ground_w, num_patches_h, num_patches_w, number_of_patches_with_ground, number_of_patches_no_ground

    # specific image img'
    for img in imgs:
        image_name = imgs[img]['file_name']
        # Check if image is from university's dataset - has no ground
        if "UNI" in image_name:
            cutGround = 2048
        else:
            cutGround = None
        # Don't use preprocessing now. We will use it in Generator when opening images. Every patch will be processed by itself.
        # That is why useLog = False
        image = openAndProcessHDRimage(img_dir + image_name, useAlpha=False, useLog=False,
                                       cutGround=cutGround)
        image_name = image_name.replace('.exr', '')
        if verbose:
            print(image_name + ' in progress')
        anns_ids = coco.getAnnIds(imgIds=imgs[img]['id'], catIds=cat_ids, iscrowd=None)
        anns = coco.loadAnns(anns_ids)
        mask = coco.annToMask(anns[0])
        for i in range(len(anns)):
            maskedAnns = coco.annToMask(anns[i])
            if (anns[i]['category_id'] == nonSkyCategory):
                # In manual annotation, label with value 2 was used to represent non-sky. Here we set it back to 0.
                mask[np.logical_and(maskedAnns > 0, mask < 2)] = nonSkyCategory  # Set to non-sky
            else:
                mask[np.logical_and(maskedAnns > 0, mask < 2)] = skyCategory  # Set to sky
        if (isBinary):
            mask = np.minimum(mask, 1)
        # Patch image and its mask
        image = np.array(image)
        mask = tf.expand_dims(mask, 0)
        mask = tf.expand_dims(mask, -1)
        image = tf.expand_dims(image, 0)
        patches_images = tf.image.extract_patches(images=image,
                                                  sizes=[1, patch_x, patch_y, 1],
                                                  strides=[1, stride, stride, 1],
                                                  rates=[1, 1, 1, 1],
                                                  padding=padding)
        del image
        patches_masks = tf.image.extract_patches(images=mask,
                                                 sizes=[1, patch_x, patch_y, 1],
                                                 strides=[1, stride, stride, 1],
                                                 rates=[1, 1, 1, 1],
                                                 padding=padding)
        del mask
        # Save patches
        for i in range(patches_images.shape[1]):
            for j in range(patches_images.shape[2]):
                if (isSplit):
                    if (count_patches in x_train):
                        # this patch should be in training set.
                        path_image_patch = os.path.join(output , 'x_train/train/')
                        path_mask_patch = os.path.join(output ,'y_train/train/')
                    else:
                        # this patch should be in validation set.
                        path_image_patch = os.path.join(output,'x_val/val/')
                        path_mask_patch = os.path.join(output ,'y_val/val/')
                # Save image patch
                single_patch_image = tf.reshape(patches_images[0, i, j], (patch_x, patch_y, 3))
                single_patch_image = np.array(single_patch_image)
                # Save as .tif file
                tifImsave(os.path.join(path_image_patch, str(count_patches) + '.tif'), single_patch_image)
                # Save mask patch
                single_patch_mask = tf.reshape(tf.reshape(patches_masks[0, i, j], (patch_x, patch_y, 1)),
                                               (patch_x, patch_y))
                single_patch_mask = np.array(single_patch_mask)
                msk = Image.fromarray(single_patch_mask)
                # Save as .png file
                msk.save(os.path.join(path_mask_patch, str(count_patches) + '.png'))
                # Preserve same name between patches of masks and images
                count_patches = count_patches + 1
                del single_patch_image, single_patch_mask, msk
        del patches_images, patches_masks
        count_exr = count_exr + 1
    if isVerboseOn:
        print(str(count_patches) + ' patches were created from ' + str(count_exr) + ' images')


### Working with Model
def getModel(BACKBONE, showModel=False):
    # define model
    model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    focal_loss = sm.losses.BinaryFocalLoss(gamma=2.0)
    model.compile('Adam', loss=focal_loss, metrics=[sm.metrics.iou_score])
    if (showModel):
        print(model.summary())
    return model


def getOldModel(model, showModel=False):
    # Define old metrics that were used in training
    # Note you can add your own metrics if other were used
    old_loss = sm.losses.bce_jaccard_loss
    focal_loss = sm.losses.BinaryFocalLoss(gamma=2.0)
    model = keras.models.load_model(model,
                                    custom_objects={'Functional': keras.models.Model,
                                                    'binary_crossentropy_plus_jaccard_loss': old_loss,
                                                    'binary_focal_loss': focal_loss, 'iou_score': sm.metrics.iou_score},
                                    compile=True)
    model.compile('Adam', loss=focal_loss, metrics=[sm.metrics.iou_score])
    if (showModel):
        print(model.summary())
    return model


# This function trains the model with generator
# data - array containing generators
def trainModelGenerator(model, data, epochs, steps_per_epoch=None, callbacks=[]):
    train_generator, val_generator = data
    if (steps_per_epoch == None):
        print("ERROR: define steps_per_epoch!")
        return
    history = model.fit(train_generator, validation_data=val_generator,
                        steps_per_epoch=steps_per_epoch, epochs=epochs, callbacks=callbacks)
    return [model, history]


# This function trains the model without generator
# data - array containing loaded split dataset
def trainModelWithoutGenerator(model, data, epochs, batch_size=None, steps_per_epoch=None, verbose=1, callbacks=[]):
    x_train, x_val, y_train, y_val = data
    # fit model
    if (batch_size):
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            validation_data=(x_val, y_val),
            callbacks=callbacks
        )
    else:
        history = model.fit_generator(train_generator, validation_data=val_generator,
                                      steps_per_epoch=steps_per_epoch,
                                      validation_steps=steps_per_epoch, epochs=epochs)
        model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            steps_per_epoch=steps_per_epoch,
            validation_steps=steps_per_epoch,
            verbose=verbose,
            validation_data=(x_val, y_val),
            callbacks=callbacks
        )
    return [model, history]


# This function was used to predict LDR input
# output - folder in which result will be saved
# input - path to the image
def predictLDR(input_path, output, patch_x, patch_y, stride, model=None, verbose=True):
    image_name = input_path.split(os.sep)[-1]  # For saving segmentation result
    image = np.array(Image.open(input_path))
    if verbose:
        printAnalyzeImage(image)
    # Save original image dimensions
    img_x = image.shape[0]
    img_y = image.shape[1]
    # Prepare image for prediction - break into 512 patches
    patched_image = patchImage(image, (patch_x, patch_y), stride, isMask=False, isNormalize=True)
    amountOfPatches = len(patched_image)
    prediction = model.predict(patched_image, verbose=1)
    del patched_image
    predicted = prediction.reshape((amountOfPatches, patch_x, patch_y))
    predicted = np.where(predicted > 0.5, 1, 0).astype('uint8')
    result = reconstructMask(predicted, [img_x, img_y], [patch_x, patch_y], stride)
    del predicted
    fileFormat = image_name.split('.')[-1]
    image_name = image_name.replace("." + fileFormat, "")
    image_name = image_name + "_segmented.png"
    if verbose:
        print("=== Saving model ===")
        print("in " + os.path.join(output, image_name))
    plt.imsave(os.path.join(output, image_name), result)
    return result


# This function merges patches back to full-sized image and considers overlapping.
# It improves prediction results, as overlapping regions are predicted several times.
def reconstructMask(patches, original_dim, patch_dim, s, method='voting'):
    i_h, i_w = original_dim
    p_h, p_w = patch_dim
    # amount of patches
    amount_h = np.floor((i_h - p_h + s) / s).astype(int)
    amount_w = np.floor((i_w - p_w + s) / s).astype(int)
    count = 0
    # Voting requires prediction to be of values 1 and 0 (uint8 type)
    result = np.zeros((i_h, i_w), dtype='uint8' if method == 'voting' else 'float32')
    voting = np.zeros((i_h, i_w), dtype='int')
    for h in range(0, amount_h):
        for w in range(0, amount_w):
            ind_h = h * s
            ind_w = w * s
            # get part of image where patch should be
            part_of_image = result[ind_h:ind_h + p_h, ind_w: ind_w + p_w]
            # get patch that corresponds to this part
            patch = np.array(patches[count])
            count += 1
            # Voting method is the only recommended method. Others do not show good results
            if method == 'voting':
                # get same patch place in voting array
                vote_result = voting[ind_h:ind_h + p_h, ind_w: ind_w + p_w]
                # where prediction was 1, increase vote for this pixel to be a sky. Otherwise, decrease
                vote_result = np.where(patch == 1, vote_result + 1, vote_result - 1)
                # Save updated patch place in voting array
                voting[ind_h:ind_h + p_h, ind_w: ind_w + p_w] = vote_result
            else:  # Not recommended
                # get part of image where patch should be
                if method == 'max':
                    part_of_image = np.maximum(part_of_image, patch)
                elif method == 'min':
                    part_of_image = np.minimum(part_of_image, patch)
                elif method == 'avg':
                    # we gradually add values of patches to original-sized image. If part of image doesn't contain patch
                    # info yet, we don't calculate average for it, but only get value of the patch
                    part_of_image = np.where(part_of_image == 0, patch, part_of_image)
                    if result.dtype == 'uint8':
                        part_of_image = (np.add(part_of_image, patch)) // 2
                    else:
                        part_of_image = (np.add(part_of_image, patch)) / 2
            result[ind_h:ind_h + p_h, ind_w: ind_w + p_w] = part_of_image
    if method == 'voting':
        # Now make final image. If pixel was voted to be sky, it will be set as sky.
        result = np.where(voting > 0, 1, 0)
    return result


# Get patches, predict, apply voting, return prediction
def predictSplitHDRImage(path_patches, path_result, model, img_x, img_y, save=True, save_name='segmented',
                         patch_x=224, patch_y=224,
                         stride=100, is_normalize=True, verbose=True):
    # Images are patched in ascending order
    test_list = [x.split('.')[0] for x in os.listdir(path_patches)]
    test_list = [int(x) for x in test_list]
    test_list = sorted(test_list)
    amount_of_patches = len(test_list)
    print(amount_of_patches)
    imgs = []
    for file in test_list:
        img_tif = tifImread(os.path.join(path_patches,str(file) + '.tif'))
        if is_normalize:
            # Process the input.
            # Note you can change this to experiment with other processing
            img_tif = apply_log_normalize(img_tif)
        imgs.append(img_tif)
        del img_tif

    imgs = np.array(imgs)
    if verbose:
        print("===Getting Data Finished===")
        print("===Prediction Started===")
    prediction = model.predict(imgs, verbose=verbose)
    del imgs
    predicted = prediction.reshape((amount_of_patches, patch_x, patch_x))
    predicted = np.where(predicted > 0.5, 1, 0).astype('uint8')
    result = reconstructMask(predicted, [img_x, img_y], [patch_x, patch_y], stride)
    if verbose:
        print("===Prediction Finished===")
    if save:
        if verbose:
            print("=== Saving result ===")
            print("in: " + os.path.join(path_result, save_name + '.png'))
        plt.imsave(os.path.join(path_result, save_name + '.png'), result)
    return result


def HdrSplit(path_exr, path_patches, patch_x=224,
             patch_y=224, stride=100, useLog=False, verbose=True):
    
    # useLog is False by default. We apply preprocessing at individual patches, not once on full picture
    image = openAndProcessHDRimage(path_exr, useAlpha=False, isTesting=False, useLog=useLog)

    if verbose:
        printAnalyzeImage(image)

    # Save original image dimensions for future use
    [img_x, img_y, _] = image.shape
    image = tf.expand_dims(image, 0)
    
    # Get filename of .exr
    # We first get filename with extension, then split it by '.' dot and get only name, ignoring extension
    filename = os.path.split(path_exr)[1].split('.')[0]

    # Create folder with patches if it doesn't exist yet
    path_patches = os.path.join(path_patches, filename + '_patches')
    if not os.path.isdir(path_patches):
        try:
            os.makedirs(path_patches)
        except OSError as error:
            print(error)
            return
    elif len(os.listdir(path_patches)) > 0:
        # Patches were already created
        return [path_patches, img_x, img_y]
    
    if verbose:
        print("==== START PREPROCESS ====")

    # Patch image
    patches_images = tf.image.extract_patches(images=image,
                                              sizes=[1, patch_x, patch_y, 1],
                                              strides=[1, stride, stride, 1],
                                              rates=[1, 1, 1, 1],
                                              padding='VALID')
    del image
    count_patches = 1
    for i in range(patches_images.shape[1]):
        for j in range(patches_images.shape[2]):
            # Save image patch
            single_patch_image = tf.reshape(patches_images[0, i, j], (patch_x, patch_y, 3))
            single_patch_image = np.array(single_patch_image)
            tifImsave(os.path.join(path_patches, str(count_patches) + '.tif'), single_patch_image)
            count_patches = count_patches + 1
            del single_patch_image
    del patches_images
    return [path_patches, img_x, img_y]


def predictHdr(input_path, output, patch_x, patch_y, stride, model=None, verbose=True):
    # Determine what is input. File or folder with splitted patches
    if '.exr' in input_path and os.path.isfile(input_path):
        # It is .exr file. We need to patch it and predict
        input_path, img_x, img_y = HdrSplit(input_path, 
                                            output, 
                                            patch_x=patch_x, 
                                            patch_y=patch_y, 
                                            stride=stride,
                                            useLog=False,
                                            verbose=verbose)

    # Load model if it was defined as a path
    if isinstance(model, str):
        # Use model user provided. Must be .h5 file
        model = keras.models.load_model(model,
                                        custom_objects={'Functional': keras.models.Model}, compile=False)
    if model==None:
        print("ERROR: Model must be defined!")
        return
    # Input should be folder with patches. If it is not, something is wrong
    if os.path.isdir(input_path) and '.tif' in os.listdir(input_path)[0]:
        # It is folder with .tif files
        result = predictSplitHDRImage(input_path, output, model, img_x=img_x, img_y=img_y, save=True,
                                      save_name='segmented',
                                      patch_x=patch_x,
                                      patch_y=patch_y,
                                      stride=stride, verbose=verbose)
    else:
        print("ERROR: Folder with patches was not found. Check provided path")
    return result


# This function tests generator for sanity
def testGenerator(generator):
    print(len(generator))  # Amount of batches in generator
    count = 0
    for i in range(0, len(generator)):
        if np.array_equiv(np.unique(generator[i][1][0]), [0., 1.]):  # Find only pairs with both sky and ground (non-sky)
            count += 1
            element = generator[i]
            plt.imshow(element[1][0])
            plt.show()
            plt.imshow(element[0][0])
            plt.show()
            print(np.unique(element[1][0]))
            if count == 3:  # Show only 3 pairs of image and mask that contain both sky and ground (non-sky)
                break

# General tools
def printMinMax(x):
    print("min max: %.4f %.4f" % (np.min(x), np.max(x)))


def showImage(x):
    plt.imshow(x)
    plt.show()

# Print some information about image and show it
def printAnalyzeImage(x):
    showImage(x)
    printMinMax(x)
    print()

# This function quickly returns desired resolutions
def getSizes(size="1080p"):
    if (size == '8K'):
        return [4096, 8192]
    if (size == '4K'):
        return [2160, 3840]
    if (size == '2K'):
        return [1080, 2048]
    if (size == '1080p'):
        return [1080, 1920]
    if (size == '720p'):
        return [720, 1280]
    if (size == '1080p'):
        return [480, 852]
    if (size == '720sq'):
        return [720, 720]
    if (size == '512sq'):
        return [512, 512]
    if (size == '256sq'):
        return [256, 256]
    if (size == '224sq'):
        return [224, 224]

# Helper function for the "countTypes"
def countTypes_subFunction(path, files):
    sky = [1]  # Only sky
    skyGround = np.array([0, 1])  # Both sky and ground (non-sky)
    skyCount = 0
    groundCount = 0
    skyGroundCount = 0
    for file in files:
        mask = np.array(Image.open(path + file))  # Open mask
        mask = np.unique(mask)  # Get only unique values in the array
        if np.array_equal(mask, skyGround):  # This mask contains both sky and ground
            skyGroundCount += 1
        elif np.array_equal(mask, sky):  # This mask contains only sky
            skyCount += 1
        else:  # This mask contains only ground
            groundCount += 1
    print('Only Sky: ', str(skyCount))
    print('Only Ground: ', str(groundCount))
    print('Both Sky and Ground (Horizon): ', str(skyGroundCount))


# This function counts amount of patches in the dataset that contain only sky, only ground or both sky and ground
# path - path to the dataset
def countTypes(path):
    # Masks already represent true values, so it is easy to count.
    # Takes some time, when large dataset
    train = path + 'y_train/train/'
    val = path + 'y_val/val/'
    files_train = os.listdir(train)
    print("==== Training set ====")
    print("Total: ", str(len(files_train)))
    countTypes_subFunction(train, files_train)

    files_val = os.listdir(val)
    print("==== Validation set ====")
    print("Total: ", str(len(files_val)))
    countTypes_subFunction(val, files_val)

In [4]:
# Set up augmentations for Albumentations library
def defineAugmentation():
    augmentation = [
        A.HorizontalFlip(p=0.5),  # Mirros the image with a probability of 50%

        # Shift, scale and rotate image in 40% of images
        A.ShiftScaleRotate(scale_limit=0.2, shift_limit=0.2, rotate_limit=25, border_mode=cv2.BORDER_REFLECT_101,
                           p=0.4),
        A.Perspective(scale=0.1, p=0.2, pad_mode=cv2.BORDER_REFLECT_101),
        # Change perspective in 20% of images. Note: Creates random mistakes when big scale
        # Apply blur in 40% of images
        A.OneOf(
            [
                A.Blur(p=1),
                A.MotionBlur(p=1),
            ],
            p=0.4,
        ),
    ]
    return A.Compose(augmentation)


# Create Custom Data Generator for HDR images.
# Reading .tif files from directory
# Expects both X (patches) and y (masks) folders
class CustomDataGen(tf.keras.utils.Sequence):

    # This function is called once the generator is initiated
    def __init__(self, X_path, y_path,
                 batch_size=16,
                 input_size=(224, 224),
                 input_channels=3,
                 shuffle=True,
                 processing='log_norm'):

        self.X_path = X_path
        self.y_path = y_path
        self.batch_size = batch_size
        self.input_size = input_size
        self.input_channels = input_channels
        self.shuffle = shuffle
        self.processing = processing
        # get list of file names without extensions. names for X and y should be the same. They are usually just numbers from 1.
        # X must have .tif extension and y .png
        self.list_files = [x.split('.')[0] for x in os.listdir(X_path)]
        self.total_length = len(self.list_files)
        self.augmentation = defineAugmentation()
        self.on_epoch_end()

    # Called once at the end of epoch
    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(self.total_length)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)  # Read input in different order

    # The function reads images and constructs X and y sets with size of batch_size
    def __data_generation(self, list_IDs_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.input_size, self.input_channels), dtype='float32')
        y = np.empty((self.batch_size, *self.input_size, 1),
                     dtype='float32')  # masks are fed into network as float32 type, yet they have only values 0.0 and 1.0
        # Fill in a batch
        for i, ID in enumerate(list_IDs_temp):
            # Open the .tif file with a patch
            image = tifImread(self.X_path + ID + '.tif')
            # Open the file with a mask. Initially it creates 2D array without any channel, so we expand the dimension.
            mask = np.expand_dims(np.array(Image.open(self.y_path + ID + '.png'), dtype='float32'), axis=-1)
            # Apply augmentation
            # The Albumentation library ensures the mask's values stay to be only 0.0 and 1.0
            output = self.augmentation(image=image, mask=mask)
            # It is important to first apply augmentations and then clip to range between 0 and 1
            # Otherwise, we would get values outside of 0...1 range, as augmentations don't consider it
            if self.processing == 'log_norm':
                X[i,] = apply_log_normalize(output['image'])  # Apply log transformation, then normalize to 0...1
            elif self.processing == 'log_tanh_norm':
                X[i,] = apply_log_tanh_normalize(
                    output['image'])  # Apply log transformation, then tanh, then normalize to 0...1
            else:
                X[i,] = output['image']  # Do not process input
            y[i,] = output['mask']  # Masks are not processed
        return X, y

    # Called when new batch of images is needed in training.
    def __getitem__(self, index):
        # X - NumPy array of patches [batch_size, patch_height, patch_width, patch_channel]
        # y - NumPy array of masks [batch_size, patch_height, patch_width]

        # Generate indexes of this new batch. We already defined order of images in on_epoch_end function and stored to self.indexes
        # Here we just get correct amount of images
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of file names (Each file name should be a number, or index)
        list_files_temp = [self.list_files[k] for k in indexes]

        # Construct X and y
        X, y = self.__data_generation(list_files_temp)

        return X, y

    # Defines amount of batches
    def __len__(self):
        return int(np.floor(self.total_length / self.batch_size))

In [6]:
### Paths
# DATA_DIR will be concatenaded with other folders provided below
DATA_DIR = 'Enter path to the folder where your datasets are located/'
# This variable is used if you plan to use already split into train/val dataset. If you only plan to create the
# dataset and split it, then this variable should point to the folder with coco .json file and folder 'images'.
# Switch the boolean 'isCreatingDatasetOn' below if that is the case.
# The folder 'patches' in the TRAIN_FOLDER will be created where split dataset will be saved.
TRAIN_FOLDER = 'Enter name of the folder with the dataset/'
# Note MODEL_FOLDER should contain 'logs' folder inside if you want to save logs
MODEL_FOLDER = 'Enter path to the folder with saved models/'
PREDICTION_PATH = 'Enter path with the image you want to predict'
PREDICTION_SAVE_FOLDER = 'Enter folder where you want to save prediciton/'
PREDICTION_NAME = 'Enter name of the prediction/' to save prediciton/'
PREDICTION_NAME = 'Enter name of the prediction/'

# Resizing initial images is not recommended, it is better to patch the image
resize = False
# Cut inital image into small patches?
patch = True
# Size of patches (or of desired resizing)
patch_x, patch_y = getSizes("224sq")
# Amount of pixels for a Step (stride) between patches. Smaller - the better.
# However, it takes more memory to create more patches.
stride = 100

### Model info
# Amount of epochs to train the model
epochs = 1
# Batch size - how many images will be loaded into network simultaniously. Bigger batch sizes require more memory
batch_size = 16
# Should network print logs?
verbose = 1
# is task - binary classification?
isBinary = True
# Backbone of the model to use for training
# For more options check here: https://github.com/qubvel/segmentation_models/blob/master/segmentation_models/backbones/backbones_factory.py
BACKBONE = 'resnet34'
# Get preprocess function of backbone. NOTE: preprocess of resnet34 does not do anything.
preprocess_input = sm.get_preprocessing(BACKBONE)
# Otherwise, if you want to use another trained model as backbone, enter its name with .h5 extension. It will be taken from the MODEL_FOLDER
# Leave this variable empty to use BACKBONE instead
OLD_MODEL_NAME = None
# Name of the file the final trained model will be saved as (without .h5 extension)
# Can be editted
MODEL_NAME = 'model_{}_{}ep_{}bch_{}X_{}Y_2'.format(BACKBONE, epochs, batch_size,
                                                    patch_x, patch_y)
### Callbacks

# Initialize tensorboard that will be saving logs of the training process. It saves loss values and IoU score.
# It creates the model logs folder in 'logs' by itself
tensorBoard = tf.keras.callbacks.TensorBoard(
    log_dir=MODEL_FOLDER + 'logs/' + BACKBONE + 'testingcode_ep' + str(epochs) + '_' + str(patch_x) + 'x' + str(
        patch_y) + '_batch' + str(batch_size) + '/fit/',
    histogram_freq=1, write_graph=True, update_freq="epoch")

# Initialize checkpoints. This callback will be saving model every time validation IoU score will be the highest so far.
# Requires disk space, every saved model of resnet34 weighs more than 200MB.

# With what name to save the checkpoints of the model?
MODEL_CHECKPOINT_NAME = MODEL_NAME + '.' + BACKBONE + '.b' + str(
    batch_size) + '.epoch{epoch:02d}-focalLoss{val_loss:.6f}-iou{val_iou_score:.6f}.h5'

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_FOLDER + MODEL_CHECKPOINT_NAME,
    verbose=1,  # Prints when checkpoint is saved
    monitor="val_iou_score",
    save_best_only=True,
    save_weights_only=False,
    mode="max")  # Save model if val_iou_score is highest so far
callbacks = [cp_callback, tensorBoard]

### Switches
isTrainingOn = True  # Train model?
isVerboseOn = True  # Output extra information while the code is executed? Good for testing
isPredictionOn = True  # Use model to make a prediction?
isCreatingDatasetOn = True  # Create new dataset before training?
useStandardDataset = False  # Load full dataset into memory, don't use generators. Not recommended. HDR is not supported!
useGeneratorDataset = True  # Use generator dataset. Supports augmentation of images
isHdr = True  # input is in HDR format (.exr)?

In [13]:
# Create new dataset
if isCreatingDatasetOn:
    # Create folder "patches"
    # Inside "patches" create folders for training and validation sets in expected format
    # Create correct split folders of training and validation sets in output path
    try:
        output = os.path.join(DATA_DIR, TRAIN_FOLDER)
        if not os.path.isdir(output):
            # Create folder with patches if it doesn't exist yet
            os.makedirs(os.path.join(output, 'patches'))
            output = os.path.join(output, 'patches')
            path_patches = os.path.join(output, 'x_train')
            os.makedirs(path_patches)
            path_patches = os.path.join(os.path.join(output, 'x_train'), 'train')
            os.makedirs(path_patches)
            path_patches = os.path.join(output, 'y_train')
            os.makedirs(path_patches)
            path_patches = os.path.join(os.path.join(output, 'y_train'), 'train')
            os.makedirs(path_patches)
            path_patches = os.path.join(output, 'x_val')
            os.makedirs(path_patches)
            path_patches = os.path.join(os.path.join(output, 'x_val'), 'val')
            os.makedirs(path_patches)
            path_patches = os.path.join(output, 'y_val')
            os.makedirs(path_patches)
            path_patches = os.path.join(os.path.join(output, 'y_val'), 'val')
            os.makedirs(path_patches)
    except OSError as error:
        raise SystemExit(error)
    path_out = os.path.join(os.path.join(DATA_DIR, TRAIN_FOLDER), 'patches')
    path_in = os.path.join(DATA_DIR, TRAIN_FOLDER)
    if isHdr:
        createInitialHDRDataset(path_in, path_out, stride=stride, patch_x=patch_x, patch_y=patch_y,
                                isSplit=True, padding='SAME', cutGround=2048, isVerboseOn=isVerboseOn)
        # Update TRAIN_FOLDER to point into split dataset
        TRAIN_FOLDER = os.path.join(os.path.join(DATA_DIR,TRAIN_FOLDER), 'patches')
    else:
        # Create LDR dataset
        all_images, all_masks = parseDatasetLDR(path_in, resize, patch_x, patch_y, isBinary, patch)
        x_train, x_val, y_train, y_val = preProcessAndSplitData([all_images, all_masks], patch_x, patch_y,
                                                                split_size=0.25, isSave=True, output=path_out)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
UNI_farm_vystice11 in progress


ResourceExhaustedError: OOM when allocating tensor with shape[1,21,82,150528] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ExtractImagePatches]

In [ ]:
# Load Training set
if isTrainingOn:
    if useStandardDataset:
        # HDR is not supported, since HDR datasets are too large to fit fully into memory
        all_images, all_masks = parseDatasetLDR(DATA_DIR + TRAIN_FOLDER, resize, patch_x, patch_y, isBinary, patch)
        x_train, x_val, y_train, y_val = preProcessAndSplitData([all_images, all_masks], patch_x, patch_y, split_size=0.25,
                                                                isSave=False)
        # Sanity check
        showImage([x_train, y_train], 3)

    if useGeneratorDataset:
        if isHdr:
            # Define custom HDR generator
            params_train = {'X_path': os.path.join(DATA_DIR, TRAIN_FOLDER , 'x_train/train/'),
                            'y_path': os.path.join(DATA_DIR , TRAIN_FOLDER , 'y_train/train/'),
                            'batch_size': batch_size,
                            'input_size': (patch_x, patch_y),
                            'input_channels': 3,
                            'shuffle': True}
            params_val = {'X_path': os.path.join(DATA_DIR , TRAIN_FOLDER , 'x_val/val/'),
                          'y_path': os.path.join(DATA_DIR , TRAIN_FOLDER , 'y_val/val/'),
                          'batch_size': batch_size,
                          'input_size': (patch_x, patch_y),
                          'input_channels': 3,
                          'shuffle': True}
            train_generator = CustomDataGen(**params_train)
            val_generator = CustomDataGen(**params_val)
            # Count how many images is in the dataset
            num_train_imgs = len(os.listdir(os.path.join(DATA_DIR , TRAIN_FOLDER ,'x_train/train/')))
            num_val_imgs = len(os.listdir(os.path.join(DATA_DIR , TRAIN_FOLDER , 'x_val/val/')))
            # Define steps per epoch the model will perform. It is based on the batch_size.
            steps_per_epoch = num_train_imgs // batch_size
            steps_per_epoch_val = num_val_imgs // batch_size
            if isVerboseOn:
                print("num_train_imgs: ", num_train_imgs)
                print("num_val_imgs: ", num_val_imgs)
                print("steps_per_epoch: ", steps_per_epoch)
                print("steps_per_epoch_val: ", steps_per_epoch_val)
        else:
            # LDR generator
            train_image_generator, mask_image_generator, valid_img_generator, valid_mask_generator = getLDRGenerator(
                DATA_DIR + TRAIN_FOLDER, batch_size)
            train_generator = zip(train_image_generator, mask_image_generator)
            val_generator = zip(valid_img_generator, valid_mask_generator)
            # Count how many images is in the dataset
            num_train_imgs = len(os.listdir(os.path.join(DATA_DIR , TRAIN_FOLDER , '/x_train/train')))
            num_val_imgs = len(os.listdir(os.path.join(DATA_DIR , TRAIN_FOLDER , '/x_val/validation')))
            # Define steps per epoch the model will perform. It is based on the batch_size.
            steps_per_epoch_val = num_val_imgs // batch_size
            steps_per_epoch = num_train_imgs // batch_size
            if isVerboseOn:
                print("num_train_imgs: ", num_train_imgs)
                print("num_val_imgs: ", num_val_imgs)
                print("steps_per_epoch: ", steps_per_epoch)
                print("steps_per_epoch_val: ", steps_per_epoch_val)

In [ ]:
# Load model
if isTrainingOn or isPredictionOn:
    if OLD_MODEL_NAME:
        # We want to train based on another model
        modelLoaded = getOldModel(os.path.join(MODEL_FOLDER , OLD_MODEL_NAME))
        if isVerboseOn:
            print('Old model loaded')
    else:
        # We want to train based on given backbone
        modelLoaded = getModel(BACKBONE)
        if isVerboseOn:
            print('Backbone ' + BACKBONE + ' loaded')

In [ ]:
# Model Training
if isTrainingOn:
    if useStandardDataset:
        # Train without generator. Not recommended
        modelTrained, history = trainModelWithoutGenerator(model=modelLoaded, data=[x_train, x_val, y_train, y_val],
                                                           epochs=epochs, batch_size=batch_size,
                                                           steps_per_epoch=steps_per_epoch,
                                                           verbose=verbose, callbacks=callbacks)
    else:
        # Train with generator.
        modelTrained, history = trainModelGenerator(model=modelLoaded, data=[train_generator, val_generator],
                                                    epochs=epochs, steps_per_epoch=steps_per_epoch,
                                                    callbacks=callbacks)

    # Save model
    modelTrained.save('{}{}.h5'.format(MODEL_FOLDER, MODEL_NAME))

In [ ]:
if isPredictionOn:
    if isTrainingOn:
        # We want to use the model that was just trained
        modelLoaded = keras.models.load_model('{}/{}.h5'.format(MODEL_FOLDER, MODEL_NAME), compile=False)
    if isHdr:
        # HDR prediction
        result = predictHdr(PREDICTION_PATH, PREDICTION_SAVE_FOLDER, patch_x, patch_y, stride, model = modelLoaded, verbose = isVerboseOn)
    else:
        # LDR prediction
        result = predictLDR(PREDICTION_PATH, PREDICTION_SAVE_FOLDER, patch_x, patch_y, stride, model = modelLoaded, verbose=isVerboseOn)